In [1]:
import os
import getpass
import datetime
import pytz
import json

from pathlib import Path

import pandas as pd
import geopandas as gpd
import shapely
from shapely_geojson import dumps

from ipyleaflet import GeoJSON, TileLayer


from traitlets import (
    Int, Bool, Unicode, link
)
import ipyvuetify as v

from sepal_ui import sepalwidgets as sw
from sepal_ui import mapping as m;
from sepal_ui.scripts import utils as su
from sepal_ui.frontend.styles import *

from component.scripts.scripts import *
from component.widget.custom_widgets import *
from component.message import cm

HTML(value='\n<style>\n.leaflet-pane {\n    z-index : 2 !important;\n}\n.leaflet-top, .leaflet-bottom {\n    z…

ResizeTrigger()

JSONDecodeError: Expecting ',' delimiter: line 14 column 9 (char 414)

In [ ]:
msg = {
    'default_api' : 'Fill and validate your Planet API key.',
    'fill_api': 'Please use a Planet API key and validate it.',
    'success_api': ('Your API key is valid', 'success'),
    'fail_api': ('Your API key is not valid', 'error'),
    'days_before': 'Search up to (days before):',
    'searching_planet': 'Searching planet imagery...',
    'max_images': 'Search up to (number of images):'
}

In [ ]:
COUNTRIES = gpd.read_file('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json')

In [ ]:
class UI(v.Layout):
    
    USER = getpass.getuser()
    TIME_SPAN = ['24 hours', '48 hours', '7 days']
    
    timespan = Unicode('24 hours').tag(sync=True)
    cloud_cover = Int(20).tag(sync=True)
    days_before = Int(0).tag(sync=True)
    max_images = Int(6).tag(sync=True)
    api_key = Unicode('').tag(sync=True)
    
    def __init__(self, **kwargs):
        
        self.class_='pa-2'

        
        super().__init__(**kwargs)
        
        # Start workspace
        self.root_dir, self.data_dir = self._workspace()
        
        self.alerts = None
        self.aoi = None
        self.aoi_alerts = None
        self.current_alert = None
        self.client = None
        
        
        self.map_ = m.SepalMap(basemaps=['Google Satellite'], dc=True)
        self.map_.show_dc()
        
        # Widgets
        
        self.w_api_key = PasswordField(
            label=cm.ui.insert_api,
            v_model=self.api_key
        )
        self.w_api_btn = sw.Btn('Check ', small=True,)
        
        self.w_spantime = v.Select(
            label="In the last",
            items=self.TIME_SPAN,
            v_model=self.timespan,
        )
        
        self.w_aoi_method = v.Select(
            label=cm.ui.aoi_method,
            v_model='Draw on map',
            items=['Draw on map', 'Select country'],
            
        )
        self.w_countries = v.Select(
            label="Select country",
            v_model='',
            items=COUNTRIES.name.to_list(),
        )
        
        self.w_alert = Alert()
        
        self.w_prev = v.Btn(
            _metadata = {'name':'previous'},
            x_small=True, 
            color="secondary",
            children=[
                v.Icon(left=True,children=['mdi-chevron-left']),
                'prev'
            ])
        
        self.w_next = v.Btn(
            _metadata = {'name' : 'next'},
            x_small=True, 
            color="secondary",
            children=[
                v.Icon(children=['mdi-chevron-right']),
                'nxt'
            ])
                            
        
        self.w_alert_list = v.Select(
            class_='ma-2',
            label='Alert', 
            items=[],
            v_model=None
        )
        
        self.w_days_before = NumberField(
            label=cm.ui.days_before,
            max_=5,
            v_model=self.days_before,
            disabled=True
        )
        
        self.w_max_images = NumberField(
            label=cm.ui.max_images,
            max_=6,
            min_=1,
            v_model=1,
            disabled=True
        )
        
        self.w_cloud_cover = v.Slider(
            label=cm.ui.cloud_cover,
            thumb_label=True,
            v_model=self.cloud_cover,
            disabled=True
        )
        
        self.w_alerts = v.Card(
            class_='pl-2 pr-2', 
            children=[
                v.Flex(
                    class_='d-flex align-center mb-2', row=True,
                    children=[
                        self.w_prev,
                        self.w_alert_list,
                        self.w_next
                    ],),
            ],
            disabled=True)
        
        self.w_state_bar = v.SystemBar(children=[''], color=sepal_darker)

        self.w_run = sw.Btn("Get Alerts")
        self.w_api_alert = Alert(children=[cm.ui.default_api], type_='info').show()
        su.hide_component(self.w_countries)
        
        # Events
        
        self.w_countries.observe(self.add_country_event, 'v_model')
        self.w_aoi_method.observe(self.aoi_method_event, 'v_model')
        self.w_alert_list.observe(self.alert_list_event, 'v_model')
        
        self.w_api_btn.on_event('click', self.validate_api_event)
        self.w_prev.on_event('click', self.prev_next_event)
        self.w_next.on_event('click', self.prev_next_event)
        
        
        self.map_.dc.on_draw(self.handle_draw)
        self.w_run.on_event('click', self._get_alerts)
        
        # Links
        
        link((self.w_api_key, 'v_model'),(self, 'api_key'))
        link((self.w_spantime, 'v_model'),(self, 'timespan'))
        link((self.w_days_before, 'v_model'),(self, 'days_before'))
        link((self.w_max_images, 'v_model'),(self, 'max_images'))
        link((self.w_cloud_cover, 'v_model'),(self, 'cloud_cover'))
        

        # View
        
        self.opt_panel = v.Card(
            class_='pa-2 mb-2',
            children=[
                v.CardTitle(children=['Alerts settings']),
                self.w_spantime,
                self.w_aoi_method,
                self.w_countries,
                self.w_run,
                self.w_alert,
            ],)
        
        self.planet_opt = v.Card(
            class_='pa-2',
            children=[
                v.CardTitle(cm.ui.planet_title),
                v.Flex(class_='d-flex align-center mb-2', 
                       row=True, 
                       children =[self.w_api_key, self.w_api_btn]
                      ),
                self.w_api_alert, 
                self.w_max_images,
                self.w_days_before,
                self.w_cloud_cover,
            ]
        )
        
        self.children = (
            
            # Left flex options panel
            v.Flex(
                xs4 =True, 
                children =[
                    self.opt_panel,
                    self.planet_opt,
            ]),
            
            # Right flex for map
            v.Flex(
                class_='ml-2', 
                xs8 = True, 
                children =[
                    self.w_alerts,
                    self.w_state_bar,
                    self.map_
            ])
        )
    
    def _toggle_planet_setts(self, on=True):
        
        if on:
            self.w_days_before.disabled = False
            self.w_cloud_cover.disabled = False
            self.w_max_images.disabled = False
            
        else:
            self.w_days_before.disabled = True
            self.w_cloud_cover.disabled = True
            self.w_max_images.disabled = True
        
    def _get_items(self):
        
        self.w_state_bar.children=[cm.ui.searching_planet]

        geom = json.loads(dumps(self.aoi_alerts.loc[self.current_alert].geometry.buffer(0.001, cap_style=3)))
        
        # Get the current year/month/day
        now = datetime.datetime.now(tz=pytz.timezone('UTC'))
        
        days_before = ([x[1] for x in list(zip(self.TIME_SPAN,[1,2,7],)) if self.timespan == x[0]])[0]
        days_before += self.days_before
        start_date = now-datetime.timedelta(days=days_before)
        print(start_date)
        req = build_request(geom, start_date, now, cloud_cover=self.cloud_cover/100)
        items = get_items('Alert', req, self.client)
        return items
    
    def _prioritize_items(self):
        
        items = self._get_items()
        items = [(item['properties']['item_type'], 
                  item['id'],
                  pd.to_datetime(item['properties']['acquired']).strftime('%Y-%m-%d-%H:%M')
                 ) for item in items[1]]
        
        items_df = pd.DataFrame(data=items, columns=['item_type', 'id', 'date'])
        items_df.sort_values(by=['item_type'])
        items_df.drop_duplicates(subset=['date'])
        
        if len(items_df):
            self.w_state_bar.children=[cm.ui.number_images.format(len(items_df))]
        else:
            self.w_state_bar.children=[cm.ui.no_planet]
        
        return items_df
        
        # Search similar
        # Clouds threshold

    def add_layers(self):
        
        items_df = self._prioritize_items()

        for i, row in items_df.iterrows():
            layer = TileLayer(
                url=f'https://tiles0.planet.com/data/v1/{row.item_type}/{row.id}/{{z}}/{{x}}/{{y}}.png?api_key={self.api_key}',
                name=f'{row.item_type}, {row.date}',
                max_zoom=15,
                attribution='Planet'
            )
            layer.__setattr__('_metadata', {'item_type':row.item_type, 'id':row.id})
            print(layer)
            if row.id not in [layer._metadata['id'] for layer in self.map_.layers if hasattr(layer, '_metadata')]:
                self.map_.add_layer(layer)
            
        
    def validate_api_event(self, widget, change, data):
        
        api_key = self.w_api_key.v_model
        
        planet_key = PlanetKey(api_key)
        self.client = planet_key.client()
        
        valid = planet_key.is_active()
        
        if valid:
            self.w_api_alert.add_msg(cm.ui.success_api.msg, cm.ui.success_api.type)
            self._toggle_planet_setts(on=True)
        else:
            self.w_api_alert.add_msg(cm.ui.fail_api.msg, cm.ui.fail_api.type)
            self._toggle_planet_setts(on=False)
            
    def remove_layers(self):
        
        # get map layers
        layers = self.map_.layers
        
        # loop and remove layers 
        [self.map_.remove_last_layer() for _ in range(len(layers))]
        
        
    def handle_draw(self, target, action, geo_json):
        
        self.remove_layers()
        if action == 'created':
            self.aoi = geo_json['geometry']
    
    def alert_list_event(self, change):
        """ Update map zoom and center when selecting an alert
        
        """
        
        # Get fire alert id
        
        self.current_alert = change['new']
        
        # Filter dataframe to get lat,lon
        
        lat = self.aoi_alerts.loc[self.current_alert]['latitude']
        lon = self.aoi_alerts.loc[self.current_alert]['longitude']
        
        self.map_.center=((lat,lon))
        self.map_.zoom=15
        
        # Search and add layers to map
        self.add_layers()
        
    def prev_next_event(self, widget, change, data):
        
        current = self.w_alert_list.v_model
        position = 0 if not current else self.w_alert_list.items.index(current)
        last = len(self.w_alert_list.items) - 1
            
        if widget._metadata['name']=='next':
            if position < last:
                self.w_alert_list.v_model = self.w_alert_list.items[position+1]

        elif widget._metadata['name']=='previous':
            if position > 0:
                self.w_alert_list.v_model = self.w_alert_list.items[position-1]
        
    def aoi_method_event(self, change):
        
        self.remove_layers()
        
        if change['new'] == 'Select country':
            self.map_.hide_dc()
            su.show_component(self.w_countries)
            
        else:
            su.hide_component(self.w_countries)
            self.map_.show_dc()
            
    
    def add_country_event(self, change):
        
        self.remove_layers()
        
        country_df = COUNTRIES[COUNTRIES['name']==change['new']]
        geometry =  country_df.iloc[0].geometry
        
        lon, lat = [xy[0] for xy in geometry.centroid.xy]
        
        data = eval(country_df.to_json())
        
        aoi = GeoJSON(data=data,
                      name=change['new'], 
                     style={'color': 'green', 'fillOpacity': 0, 'weight': 3})
            
        
        self.aoi = aoi.data['features'][0]['geometry']
        
        min_lon, min_lat, max_lon, max_lat = geometry.bounds

        # Get (x, y) of the 4 cardinal points
        tl = (max_lat, min_lon)
        bl = (min_lat, min_lon)
        tr = (max_lat, max_lon)
        br = (min_lat, max_lon)
        
        self.map_.zoom_bounds([tl,bl, tr, br])
        self.map_.center = (lat, lon)
        self.map_.add_layer(aoi)


    def validate_inputs(self):
        
        if not self.aoi:
            self.w_alert.add_msg(cm.ui.valid_aoi,type_='error')
            self.restore_widgets()

            raise
    
    def restore_widgets(self):
        
        self.w_run.disabled=False
        self.w_run.loading=False
        self.w_alert_list.items = []
        self.w_alert_list.v_model = None

    def _get_url(self, satellite):
        
        satellites = {
            'viirs': ('SUOMI_VIIRS_C2', 'suomi-npp-viirs-c2'),
            'modis': ('MODIS_C6', 'c6'),
            'viirsnoa': ('J1_VIIRS_C2', 'noaa-20-viirs-c2'),
        
        }
        
        sat = satellites[satellite]
        timespan = self.timespan.replace(' hours', 'h').replace(' days','d')
        
        url=f"https://firms.modaps.eosdis.nasa.gov/data/active_fire/{sat[1]}/csv/{sat[0]}_Global_{timespan}.csv"
        return url
        
    def _get_alerts(self, widget, change, data):
        
        self.validate_inputs()
        widget.toggle_loading()
        
        self.w_alert.add_live_msg(cm.ui.downloading_alerts, type_='info')

        
        url = self._get_url('viirs')
        confidence=None
        
        df = pd.read_csv(url)
        alerts_gdf = gpd.GeoDataFrame(df, 
                                      geometry=gpd.points_from_xy(df.longitude, 
                                                                  df.latitude), 
                                      crs="EPSG:4326")
        
        if confidence: alerts_gdf = alerts_gdf[alerts_gdf.confidence==confidence]
        
        self.alerts = alerts_gdf
        
        self.aoi_alerts = self._clip_to_aoi()

        self.w_alert.add_msg(cm.ui.alert_number, type_='success')
        
        alert_list_item = list(self.aoi_alerts.index)
        self.w_alert_list.items = alert_list_item
        
        json_aoi_alerts = eval(self.aoi_alerts.to_json())
        json_aoi_alerts = GeoJSON(data=json_aoi_alerts,
                                name='Alerts', 
                                point_style={
                                     'radius': 2, 
                                     'color': 
                                     'red', 
                                     'fillOpacity': 0.1, 
                                     'weight': 2
                                 },
                                hover_style={
                                    'color': 'white', 
                                    'dashArray': '0', 
                                    'fillOpacity': 0.5
                                },)
        
        
        self.map_.add_layer(json_aoi_alerts)
        self.w_alerts.disabled = False
        
        
        widget.toggle_loading()
    
    def _clip_to_aoi(self):
        
        # Clip alerts_gdf to the selected aoi
        ""
        self.w_alert.add_live_msg(msg=cm.ui.clipping,type_='info')
        
        clip_geometry = shapely.geometry.Polygon(self.aoi['coordinates'][0])
        
        alerts = self.alerts[self.alerts.geometry.intersects(clip_geometry)]
        
        return alerts
    
    def _workspace(self):
        """ Creates the workspace necessary to store and manipulate the module

        return:
            Returns environment Paths

        """

        base_dir = Path(os.path.join('/home', self.USER))

        root_dir = base_dir/'Planet_fire_explorer'
        data_dir = root_dir/'data'

        root_dir.mkdir(parents=True, exist_ok=True)
        data_dir.mkdir(parents=True, exist_ok=True)

        return root_dir, data_dir

In [ ]:
ui = UI()

In [ ]:
ui